In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
import torchvision.datasets as datasets

# Define transformation to convert images to tensors and normalize
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


### airplanes and birds (Class 0, 2)

In [3]:
from torch.utils.data import DataLoader, Subset
from sklearn import svm
from sklearn.metrics import accuracy_score

In [14]:
# Filter for airplane (0) and bird (2) classes
classes_to_keep = [0, 2]
train_indices = [i for i, label in enumerate(train_dataset.targets) if label in classes_to_keep]
test_indices = [i for i, label in enumerate(test_dataset.targets) if label in classes_to_keep]

train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(test_dataset, test_indices)

# Create data loaders
train_loader = DataLoader(train_subset, batch_size=len(train_subset), shuffle=False)
test_loader = DataLoader(test_subset, batch_size=len(test_subset), shuffle=False)

# Extract data
x_train, y_train = next(iter(train_loader))
x_test, y_test = next(iter(test_loader))

# Flatten images into vectors
x_train_flat = x_train.view(x_train.size(0), -1).numpy()
x_test_flat = x_test.view(x_test.size(0), -1).numpy()

# Normalize pixel values
x_train_flat = x_train_flat / 255.0
x_test_flat = x_test_flat / 255.0

# Relabel classes to 0 and 1 for easier interpretation
y_train = y_train.numpy()
y_test = y_test.numpy()
y_train_binary = (y_train == 2).astype(int)  # 0: airplane, 1: bird
y_test_binary = (y_test == 2).astype(int)

In [15]:
print(x_train_flat.shape)

(10000, 3072)


In [24]:
import random

indices = random.sample(range(1, 10001), 2000)
x_ss = x_train_flat[indices,]
y_ss = y_train[indices]

print(x_ss.shape)
print(y_ss.shape)

(2000, 3072)
(2000,)


In [25]:
# Initialize and train SVM
clf = svm.SVC(kernel='linear')
clf.fit(x_ss,y_ss)

# Predict and evaluate
y_pred = clf.predict(x_test_flat)
accuracy = accuracy_score(y_test_binary, y_pred)

print(f"SVM .2 random subset accuracy on airplane vs bird: {accuracy * 100:.2f}%")

SVM .2 random subset accuracy on airplane vs bird: 39.05%
